This notebook demonstrates how to use OptKeras, wrapper of Keras and Optuna

##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    # Specify a directory in Google Drive
    dir = '/content/drive/My Drive/Colab Notebooks/OptKeras'
    os.chdir(dir)    
    # Check the environment info
    print('## Current working directory: ', os.getcwd())
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
## Current working directory:  /content/drive/My Drive/Colab Notebooks/OptKeras
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.119208 hours (429.15 sec)
## Check the GPU info
Sun Feb 17 03:18:04 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+-----------------

## Install Optuna 0.7.0

In [2]:
!pip install optuna==0.7.0

    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 81kB 5.3MB/s 
    100% |████████████████████████████████| 51kB 22.4MB/s 
    100% |████████████████████████████████| 92kB 27.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/97/c7/868e9cd5d6a23cfb98266f30d59131e5bd441132bdf8da18d5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna pyperclip


## Install the latest version of OptKeras

In [3]:
!pip install git+https://github.com/Minyus/optkeras.git

  Cloning https://github.com/Minyus/optkeras.git to /tmp/pip-req-build-pmhu1fw3
  Stored in directory: /tmp/pip-ephem-wheel-cache-a91gxl1u/wheels/ad/c8/9d/67471f9fd7c5fd15961a2e7aa3d15215212d0631ea96df8b5c
Successfully built optkeras


## Import modules

In [4]:
# import Keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

import numpy as np
import pandas as pd

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.0-rc1
Optuna 0.7.0
OptKeras 0.0.1


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST', 'MNIST_1000samples']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 2s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## A simple Keras model

In [7]:
model = Sequential()
model.add(Conv2D(
    filters = 32, 
    kernel_size = 3, 
    strides = 1,
    activation = 'relu', 
    input_shape = input_shape ))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer = Adam(), 
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])			  
model.fit(x_train, y_train, 
          validation_data = (x_test, y_test), shuffle = True,
          batch_size = 512, epochs = 2) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 15s 253us/step - loss: 0.4386 - acc: 0.8816 - val_loss: 0.2121 - val_acc: 0.9417
Epoch 2/2
60000/60000 [==============================] - 2s 31us/step - loss: 0.1713 - acc: 0.9521 - val_loss: 0.1256 - val_acc: 0.9659


## Optimization of a simple Keras model without pruning

In [8]:
study_name = dataset_name + '_Simple'

ok = OptKeras(study_name=study_name)

optuna.logging.set_verbosity(optuna.logging.WARN)

def objective(trial):
   
    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # [Required] Specify callbacks(trial) in fit method
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, n_trials = 16, timeout = 12*60*60)

[2019-02-17 03:19:00.137518] Ready for optimization. (message printed as verbose is set to 1+)
[2019-02-17 03:19:00.456854] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-17 03:19:06.341564] Latest trial id: 0, value: 0.028299999523162866 (TrialState.COMPLETE) | Best trial id: 0, value: 0.028299999523162866, parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'relu'}
[2019-02-17 03:19:09.671199] Latest trial id: 1, value: 0.0792999995231628 (TrialState.COMPLETE) | Best trial id: 0, value: 0.028299999523162866, parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'relu'}
[2019-02-17 03:19:12.173860] Latest trial id: 2, value: 0.08070000066757199 (TrialState.COMPLETE) | Best trial id: 0, value: 0.028299999523162866, parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'relu'}
[2019-02-17 03:19:14.306053] Latest trial id: 3, value: 0.08340000076293941 (TrialState.COMPLE

## Grid Search of a simple Keras model

In [9]:
study_name = dataset_name + '_GridSearch'

ok = OptKeras(study_name=study_name, grid_search_mode=True)

optuna.logging.set_verbosity(optuna.logging.WARN)

def objective(trial):
   
    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # [Required] Specify callbacks(trial) in fit method
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for grid_search by Optuna
ok.grid_search(objective, n_trials = 2*2*2*2, timeout = 12*60*60)

[2019-02-17 03:20:06.348352] Ready for optimization. (message printed as verbose is set to 1+)
[2019-02-17 03:20:06.465484] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-17 03:20:11.454800] Completed:   6% (    1 /    16)
[2019-02-17 03:20:11.557470] Latest trial id: 0, value: 0.07819999942779543 (TrialState.COMPLETE) | Best trial id: 0, value: 0.07819999942779543, parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'linear'}
[2019-02-17 03:20:13.604243] Completed:  12% (    2 /    16)
[2019-02-17 03:20:13.707633] Latest trial id: 1, value: 0.05769999990463259 (TrialState.COMPLETE) | Best trial id: 1, value: 0.05769999990463259, parameters: {'filters': 32, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-02-17 03:20:18.397450] Completed:  19% (    3 /    16)
[2019-02-17 03:20:18.502337] Latest trial id: 2, value: 0.07899999961853033 (TrialState.COMPLETE) | Best trial id: 1, value: 0.0576999999046

## Optimization of a Keras model with pruning

In [10]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=optuna.samplers.TPESampler(
        consider_prior=True, prior_weight=1.0, 
        consider_magic_clip=True, consider_endpoints=False, 
        n_startup_trials=10, n_ei_candidates=24, 
        seed=None), 
    pruner=optuna.pruners.SuccessiveHalvingPruner(
        min_resource=1, reduction_factor=4, min_early_stopping_rate=0), 
    study_name = study_name, 
    direction='minimize', 
    load_if_exists = True,
    # parameters for OptKeras
    monitor='val_error', # Either 'val_error' (1 - val_acc) or 'val_loss'
    enable_pruning=True, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1 )

optuna.logging.set_verbosity(optuna.logging.WARN)

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters = int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    #batch_size = int(trial.suggest_discrete_uniform(
    #                  'Batch_size', 256, 512, 256) )
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size = batch_size,
                  epochs = epochs, validation_data = (x_test, y_test),
                  shuffle = True,
                  callbacks = ok.callbacks(trial), 
                  verbose = ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks = ok.callbacks(trial), 
                            verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, n_trials=None, timeout = 0.5*60*60)

[2019-02-17 03:22:18.950760] Ready for optimization. (message printed as verbose is set to 1+)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-02-17 03:22:19.762612] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-17 03:22:44.720062] Latest trial id: 1, value: 0.020000000000000018 (TrialState.COMPLETE) | Best trial id: 1, value: 0.020000000000000018, parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.0, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}
[2019-02-17 03:25:36.284717] Latest trial id: 2, value: 0.010299999999999976 (TrialState.COMPLETE) | Best trial id: 2, value: 0.010299999999999976, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_

## Check the results

In [11]:
# OptKeras best_trial returns 
print('Best trial id: ', ok.best_trial.trial_id)
print('Best value:', ok.best_trial.value)
print('Best params: ')
ok.best_trial.params

Best trial id:  14
Best value: 0.007600000000000051
Best params: 


{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.25,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters': 64.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.25,
 'FC_1_num_hidden_units': 256.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 0,
 'FC_2_dropout_rate': 0.25,
 'FC_2_num_hidden_units': 512.0,
 'Optimizer': 'Adam'}

In [12]:
"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
study = ok.study
study.best_trial.params 

{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.25,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters': 64.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.25,
 'FC_1_num_hidden_units': 256.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 0,
 'FC_2_dropout_rate': 0.25,
 'FC_2_num_hidden_units': 512.0,
 'Optimizer': 'Adam'}

In [13]:
## Check the Optuna CSV log file 
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
pd.read_csv(ok.optuna_log_file_path)

Data Frame read from MNIST_Optimized_Optuna.csv 



,trial_id,state,value,datetime_start,datetime_complete,Batch_size,Conv,Conv_1_dropout_rate,Conv_1_kernel_size,Conv_1_max_pooling_size,...,intermediate_values_0,intermediate_values_1,intermediate_values_2,intermediate_values_3,intermediate_values_4,intermediate_values_5,intermediate_values_6,intermediate_values_7,intermediate_values_8,intermediate_values_9
0,1,TrialState.COMPLETE,0.0200,2019-02-17 03:22:18.962472,2019-02-17 03:22:43.735627,256,0,NaN,NaN,NaN,...,0.0392,0.0292,0.0279,0.0252,0.0214,0.0193,0.0196,0.0207,0.0214,0.0200
1,2,TrialState.COMPLETE,0.0103,2019-02-17 03:22:43.771368,2019-02-17 03:25:35.289416,256,1,0.5,2.0,2.0,...,0.0250,0.0190,0.0157,0.0122,0.0114,0.0114,0.0115,0.0118,0.0106,0.0103
2,3,TrialState.COMPLETE,0.0103,2019-02-17 03:25:35.328963,2019-02-17 03:26:18.581397,256,1,0.5,3.0,2.0,...,0.0201,0.0156,0.0126,0.0128,0.0105,0.0119,0.0109,0.0101,0.0096,0.0103
3,4,TrialState.COMPLETE,0.0087,2019-02-17 03:26:18.624491,2019-02-17 03:29:06.782529,256,1,0.5,3.0,2.0,...,0.0153,0.0156,0.0171,0.0103,0.0096,0.0081,0.0087,0.0081,0.0081,0.0087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,33,TrialState.PRUNED,0.0173,2019-02-17 03:50:08.367605,2019-02-17 03:50:19.196433,256,1,0.0,2.0,3.0,...,0.0338,0.0173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,34,TrialState.PRUNED,0.0097,2019-02-17 03:50:19.231568,2019-02-17 03:51:46.539676,256,1,0.0,3.0,2.0,...,0.0196,0.0153,0.0119,0.0140,0.0097,NaN,NaN,NaN,NaN,NaN
34,35,TrialState.PRUNED,0.0315,2019-02-17 03:51:46.582991,2019-02-17 03:51:55.964161,256,0,NaN,NaN,NaN,...,0.0461,0.0315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,36,TrialState.PRUNED,0.0242,2019-02-17 03:51:56.012677,2019-02-17 03:52:30.565889,256,0,NaN,NaN,NaN,...,0.0328,0.0242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
## Check the Keras CSV log file

pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
pd.read_csv(ok.keras_log_file_path)

Data Frame read from MNIST_Optimized_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,2019-02-17 03:22:20.750218,2019-02-17 03:22:23.355337,val_error,1,0.929817,0.070183,0.228616,0.9608,0.0392,0.125338
1,1,2019-02-17 03:22:23.845494,2019-02-17 03:22:25.771266,val_error,1,0.973300,0.026700,0.085822,0.9708,0.0292,0.096068
2,2,2019-02-17 03:22:26.091224,2019-02-17 03:22:28.035295,val_error,1,0.983083,0.016917,0.055466,0.9721,0.0279,0.088066
3,3,2019-02-17 03:22:28.341721,2019-02-17 03:22:30.278312,val_error,1,0.988050,0.011950,0.039195,0.9748,0.0252,0.080062
...,...,...,...,...,...,...,...,...,...,...,...
131,0,2019-02-17 03:51:49.455350,2019-02-17 03:51:52.219222,val_error,35,0.904183,0.095817,0.314823,0.9539,0.0461,0.145614
132,1,2019-02-17 03:51:53.110567,2019-02-17 03:51:55.198372,val_error,35,0.954833,0.045167,0.146437,0.9685,0.0315,0.099852
133,0,2019-02-17 03:51:58.102622,2019-02-17 03:52:13.367637,val_error,36,0.891176,0.108824,0.373676,0.9672,0.0328,0.110411
134,1,2019-02-17 03:52:14.547104,2019-02-17 03:52:28.910951,val_error,36,0.958456,0.041544,0.137814,0.9758,0.0242,0.077835
